# Model Selection

In [ ]:
import psycopg2
from psycopg2 import sql

from typing import List, Tuple
import logging
from itertools import repeat, chain
import re

from tqdm import tqdm

import numpy as np
import pandas as pd

import tables

# from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline

import lightgbm as lgbm

# from ray import tune
import optuna

## Load data

Get the training and validation datasets from an SQL database

In [ ]:
db_query = sql.SQL("""SELECT ttv.id, tweets.retweet_count, tweets.like_count, tweets.reply_count,
    
                        tweets.video AS tweet_has_video,
                        tweets.photo AS tweet_has_photo,

                        articles.video AS article_has_video,
                        articles.audio AS article_has_audio,
                        articles.comments,

                        textlengths.tweetlength,
                        textlengths.titlelength,
                        textlengths.summarylength,
                        textlengths.articlelength,

                        sections.section,

                        timeinfo.seconds,
                        timeinfo.month,
                        timeinfo.dayofweek,
                        
                        sentiment.vader_tweet_texts_neg,
                        sentiment.vader_tweet_texts_neu,
                        sentiment.vader_tweet_texts_pos,
                        sentiment.vader_tweet_texts_compound,
                        sentiment.vader_article_titles_neg,
                        sentiment.vader_article_titles_neu,
                        sentiment.vader_article_titles_pos,
                        sentiment.vader_article_titles_compound,
                        sentiment.vader_article_summaries_neg,
                        sentiment.vader_article_summaries_neu,
                        sentiment.vader_article_summaries_pos,
                        sentiment.vader_article_summaries_compound,
                        sentiment.vader_article_main_neg,
                        sentiment.vader_article_main_neu,
                        sentiment.vader_article_main_pos,
                        sentiment.vader_article_main_compound,

                        sentiment.distilbert_tweet_texts_negative,
                        sentiment.distilbert_tweet_texts_positive,
                        sentiment.distilbert_article_titles_negative,
                        sentiment.distilbert_article_titles_positive,
                        sentiment.distilbert_article_summaries_negative,
                        sentiment.distilbert_article_summaries_positive,
                        sentiment.distilbert_article_main_negative,
                        sentiment.distilbert_article_main_positive,

                        sentiment.roberta_tweet_texts_negative,
                        sentiment.roberta_tweet_texts_positive,
                        sentiment.roberta_tweet_texts_neutral,
                        sentiment.roberta_article_titles_negative,
                        sentiment.roberta_article_titles_positive,
                        sentiment.roberta_article_titles_neutral,
                        sentiment.roberta_article_summaries_negative,
                        sentiment.roberta_article_summaries_positive,
                        sentiment.roberta_article_summaries_neutral,
                        sentiment.roberta_article_main_negative,
                        sentiment.roberta_article_main_positive,
                        sentiment.roberta_article_main_neutral,

                        sentiment.siebert_tweet_texts_negative,
                        sentiment.siebert_tweet_texts_positive,
                        sentiment.siebert_article_titles_negative,
                        sentiment.siebert_article_titles_positive,
                        sentiment.siebert_article_summaries_negative,
                        sentiment.siebert_article_summaries_positive,
                        sentiment.siebert_article_main_negative,
                        sentiment.siebert_article_main_positive

                    FROM (SELECT id FROM traintest WHERE split = {}) AS ttv
                    INNER JOIN tweets ON ttv.id = tweets.id
                    INNER JOIN articles ON ttv.id = articles.id
                    INNER JOIN textlengths ON ttv.id = textlengths.id
                    INNER JOIN sections ON ttv.id = sections.id
                    INNER JOIN timeinfo ON ttv.id = timeinfo.id
                    INNER JOIN sentiment ON ttv.id = sentiment.id
                    WHERE tweets.date < {}
                    ;"""
)

In [ ]:
CUTOFF_DATE = '2022-04-16'

with psycopg2.connect(host = 'localhost', database = 'nytpopular') as conn:
    with conn.cursor() as cursor:
        cursor.execute(db_query.format(sql.Literal('train'), sql.Literal(CUTOFF_DATE)))
        train = cursor.fetchall()
        train_column_names = [description[0] for description in cursor.description]
        cursor.execute(db_query.format(sql.Literal('valid'), sql.Literal(CUTOFF_DATE)))
        val = cursor.fetchall()
        val_column_names = [description[0] for description in cursor.description]
        cursor.execute(db_query.format(sql.Literal('test'), sql.Literal(CUTOFF_DATE)))
        test = cursor.fetchall()
        test_column_names = [description[0] for description in cursor.description]

In [ ]:
def transform_target(target : pd.Series) -> pd.Series:
    return np.log10(target + 1.0)

def preprocess(dataset : List[Tuple], columns : List[str]) -> pd.DataFrame:
    X = pd.DataFrame(dataset, columns = columns).set_index('id')
    likes = transform_target(X['like_count'])
    retweets = transform_target(X['retweet_count'])
    replies = transform_target(X['reply_count'])
    X = X.drop(['like_count', 'retweet_count', 'reply_count'], axis = 1)
    for key in ['tweet_has_video', 'tweet_has_photo', 'article_has_video', 'article_has_audio']: # 0/1 encode these bools
        X[key] = X[key].astype(int)
    # Were comments enabled? The number of comments cannot be kept as a feature because this is not information that is available before an article is published.
    # It's possible that 
    X['comments'] = (X['comments']/X['comments']).fillna(0).astype(int)
    return X, likes, retweets, replies

In [ ]:
def remove_rare_sections(X): # Danger: intentionally mutates the input!!!
    section_counts = X['section'].value_counts()
    MIN_COUNT = 2
    sections_to_be_removed = section_counts[section_counts <= MIN_COUNT].index
    X.loc[X['section'].isin(sections_to_be_removed), 'section'] = np.nan

In [ ]:
X_train, likes_train, retweets_train, replies_train = preprocess(train, train_column_names)
X_val, likes_val, retweets_val, replies_val = preprocess(val, val_column_names)
X_test, likes_test, retweets_test, replies_test = preprocess(test, test_column_names)

In [ ]:
remove_rare_sections(X_train)

## Baseline Metrics

In [ ]:
# X_train_val_combined = pd.concat([X_train, X_val])
X_combined, likes_combined, retweets_combined, replies_combined = preprocess([*train, *val], train_column_names)

In [ ]:
for strategy in ['mean', 'median']:
    for label, y_combined, y_test in zip(['likes', 'retweets', 'replies'], [likes_combined, retweets_combined, replies_combined], [likes_test, retweets_test, replies_test]):
        baseline = DummyRegressor(strategy=strategy)
        baseline.fit(X_combined, y_combined)
        print(f'R^2 for the {strategy} model for {label} is {baseline.score(X_test, y_test)}.') # Should be near 0, since R^2 compares the model to the mean model (i.e. exactly zero for the training data).
    print()

# A little surprising to me that the median model is actually slightly worse than the mean model.

## Data pipeline

Pipeline transformers for selecting features and getting the data into the correct format.

I will use LightGBM for tree-based gradient boosting because LightGBM is accurate, fast, handles categorical variables without needed to one-hot encode, and handles missing data (although how missing data is handled is [not always intelligent](https://github.com/microsoft/LightGBM/issues/2921).

Things to watch out for:
- Although LightGBM can handle categorical features in pandas DataFrames, this is potentially dangerous if new unseen categories appear after training. I will use scikit-learn to encode categorical data into integers.
- Although multicollinearity will not significantly adversely affect predictions, it may make interpretation difficult.

In [ ]:
class IntegerCategoricalEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y = None):
        # encoded_missing_value is new in scikit-learn 1.1. Remove it for older versions.
        self._ordinalencoder = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = np.nan, encoded_missing_value = np.nan)
        self._ordinalencoder.fit(X[[self.column_name]])
        return self

    def transform(self, X):
        X = X.copy()
        X[self.column_name] = self._ordinalencoder.transform(X[[self.column_name]])
        return X

In [ ]:
DAILY_SECONDS = 86400
HOURLY_SECONDS = 3600

def gaussian(x, center, width):
    return np.exp(-(x - center)**2 / (2 * width **2))

def periodic_gaussian(x, center_in_hours, width_in_hours): # Not actually periodic
    center = center_in_hours * HOURLY_SECONDS
    width = width_in_hours * HOURLY_SECONDS
    return gaussian(x - DAILY_SECONDS, center, width) + gaussian(x, center, width) + gaussian(x + DAILY_SECONDS, center, width)

In [ ]:
class TimeEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, time_encode_type : str, n_time_gaussians = None):
        if time_encode_type == 'raw':
            assert n_time_gaussians is None, 'n_time_gaussians specified when time_encode_type == "raw"'
        elif time_encode_type == 'rbf': # Radial Basis Functions
            assert n_time_gaussians in [4, 6, 12, 24], 'n_time_gaussians must divide 24'
            self.n_time_gaussians = n_time_gaussians
        else:
            assert False, 'Unknown time_encode_type'
        self.time_encode_type = time_encode_type
    
    def fit(self, X, y = None):
        return self

    def transform(self, X):
        if self.time_encode_type == 'rbf':
            seconds = X['seconds']
            X = X.drop(['seconds'], axis = 1)
            for basis_index in range(self.n_time_gaussians):
                hour = basis_index * 24 // self.n_time_gaussians
                X[f'hour_{hour}'] = periodic_gaussian(seconds, hour, 24 // self.n_time_gaussians) # A measure of how far away the time is from the "hour". ~1 represents right on the hour, ~0 represents far away from the hour.
        return X

In [ ]:
class SentimentSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, sentiment_type):
        self._sentiment_models = {'none', 'vader', 'distilbert', 'roberta', 'siebert'}
        assert sentiment_type in self._sentiment_models, 'Unknown sentiment analyzer'
        self.sentiment_type = sentiment_type

    def fit(self, X, y = None):
        return self

    def transform(self, X):
        unwanted_columns = [column for column in X.columns if any(sentiment_type in column for sentiment_type in self._sentiment_models.difference({self.sentiment_type}))]
        X = X.drop(unwanted_columns, axis = 1)
        return X

In [ ]:
class ColumnRemover(BaseEstimator, TransformerMixin):
    
    def __init__(self, drop_month = False, drop_dayofweek = False, drop_comments = False, drop_lengths = False):
        self.drop_month = drop_month
        self.drop_dayofweek = drop_dayofweek
        self.drop_comments = drop_comments
        self.drop_lengths = drop_lengths

    def fit(self, X, y = None):
        return self

    def transform(self, X):
        unwanted_columns = []
        if self.drop_month:
            unwanted_columns.append('month')
        if self.drop_dayofweek:
            unwanted_columns.append('dayofweek')
        if self.drop_comments:
            unwanted_columns.append('comments')
        if self.drop_lengths:
            unwanted_columns.extend([column for column in X.columns if 'length' in column])
        X = X.drop(unwanted_columns, axis = 1)
        return X

In [ ]:
_nTopics_list = list(chain(range(5,30,5),range(30, 200 + 1, 10)))
_valid_nTopics = set(_nTopics_list)

def load_topic_df(filename : str, nTopics : int, top_topics : bool = False, include_scores = False, MAX_TOPICS = 5) -> pd.DataFrame:
    with tables.open_file(filename, mode = 'r') as f:
        table = f.root.topic[f'ntopics{nTopics}']
        ids, topicvectors = zip(*[(x['id'], x['topicvector']) for x in table.iterrows()])
        if top_topics:
            topic_ranks = [(-tv).argsort()[:MAX_TOPICS] for tv in topicvectors]
            if include_scores:
                # sorted_topic_scores = [np.sort(tv)[::-1] for tv in topicvectors]
                sorted_topic_scores = [[*tr, *tv[tr]] for tr, tv in zip(topic_ranks, topicvectors)]
                return pd.DataFrame(sorted_topic_scores, index = ids, columns = [f'best_topic_{n:03}' for n in range(MAX_TOPICS)] + [f'best_topic_scores_{n:03}' for n in range(MAX_TOPICS)])
            else:
                return pd.DataFrame(topic_ranks, index = ids, columns = [f'best_topic_{n:03}' for n in range(MAX_TOPICS)])
        else:
            return pd.DataFrame(topicvectors, index = ids, columns = [f'topic_{n:03}' for n in range(nTopics)])

class TopicLoader(BaseEstimator, TransformerMixin):

    def __init__(self, nTopics : int, top_5 : bool = False, include_scores : bool = False, symmetricbeta : bool = False):
        assert nTopics in _valid_nTopics, 'Invalid number of topics'
        assert (not include_scores) or top_5, 'include_scores provided when top_5 is False'
        self.nTopics = nTopics
        self.top_5 = top_5
        self.include_scores = include_scores
        self.symmetricbeta = symmetricbeta
        if symmetricbeta:
            filename = r'GensimModels/article_data_symmetricbeta.h5'
        else:
            filename = r'GensimModels/article_data.h5'
        self.topics_df = load_topic_df(filename, nTopics, top_5, include_scores)

    def fit(self, X, y = None):
        return self

    def transform(self, X):
        return pd.merge(X, self.topics_df, how = 'inner', left_index = True, right_index = True)

In [ ]:
best_topic_regex = re.compile(r'^best_topic_\d{3}$')

def categorical_identifier(label : str) -> bool:
    if label in {'section', 'month', 'dayofweek', 'comments', 'article_has_audio', 'article_has_video', 'tweet_has_video', 'tweet_has_photo'}:
        return True
    elif best_topic_regex.match(label):
        return True
    else:
        return False

## Hyperparameter Tuning

In [ ]:
# TO DO: Fix code duplication

In [ ]:
lgbm_logger = logging.getLogger('lgbm')
loghandle = logging.FileHandler(f'TreeModels/logs/lgbm.log')
logformat = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
loghandle.setFormatter(logformat)
lgbm_logger.addHandler(loghandle)
lgbm_logger.setLevel(logging.INFO)

lgbm.register_logger(lgbm_logger)

In [ ]:
def objective_factory(X_train, y_train, X_val, y_val, evaluate = False, test_set = None, n_estimators = None):

    def objective(trial : optuna.Trial):

        TIME_ENCODE_TYPE = trial.suggest_categorical('time_encode_type', ['raw', 'rbf'])
        if TIME_ENCODE_TYPE == 'rbf':
            N_TIME_GAUSSIANS = [4, 6, 12, 24][trial.suggest_int('n_time_gaussians_INDEX', 0, 3)]
        else:
            N_TIME_GAUSSIANS = None
        SENTIMENT_TYPE = trial.suggest_categorical('sentiment_type', ['vader', 'distilbert', 'roberta', 'siebert'])
        NTOPICS = _nTopics_list[trial.suggest_int('nTopics_INDEX', 0, len(_nTopics_list) - 1)]
        TOP_5 = trial.suggest_categorical('top_5', [True, False])
        if TOP_5:
            INCLUDE_SCORES = trial.suggest_categorical('include_scores', [True, False])
        else:
            INCLUDE_SCORES = None
        SYMMETRICBETA = trial.suggest_categorical('symmetricbeta', [True, False])

        pipe = Pipeline([
            ('section_encoder', IntegerCategoricalEncoder('section')),
            ('time_encoder', TimeEncoder(time_encode_type = TIME_ENCODE_TYPE, n_time_gaussians = N_TIME_GAUSSIANS)),
            ('sentiment_selector', SentimentSelector(sentiment_type = SENTIMENT_TYPE)),
            ('topic_loader', TopicLoader(nTopics = NTOPICS, top_5 = TOP_5, include_scores = INCLUDE_SCORES, symmetricbeta = SYMMETRICBETA))
        ])

        X_t = pipe.fit_transform(X_train)
        X_v = pipe.fit_transform(X_val)

        columns = list(X_t.columns)
        cats = [idx for idx, col in enumerate(columns) if categorical_identifier(col)]

        regressor_params = {
            'device_type' : 'cpu',
            'objective' : 'regression',
            'n_estimators' : 999_999 if n_estimators is None else n_estimators,
            'learning_rate' : 0.02,
            'num_leaves' : trial.suggest_int('num_leaves', 4, 2000),
            'max_depth' : trial.suggest_int('max_depth', 2, 30),
            'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 10, 200, step = 10),
            'lambda_l1' : trial.suggest_float('lambda_l1', 1e-6, 10.0, log = True),
            'lambda_l2' : trial.suggest_float('lambda_l2', 1e-6, 10.0, log = True),
            'min_gain_to_split' : trial.suggest_float('min_gain_to_split', 1e-6, 1.0, log = True),
            'bagging_freq' : 1,
            'bagging_fraction' : trial.suggest_float('bagging_fraction', 0.1, 0.99),
            'feature_fraction' : trial.suggest_float('feature_fraction', 0.05, 1.0),
            'min_data_per_group' : trial.suggest_int('min_data_per_group', 1, 10),
            'cat_smooth' : trial.suggest_float('cat_smooth', 0, 25.0),
            'n_jobs': 6,
            'importance_type' : 'gain'
        }
        # if regressor_params['bagging_freq'] != 0:
        #     regressor_params['bagging_fraction'] = trial.suggest_float('bagging_fraction', 0.1, 0.99)

        callbacks = [lgbm.log_evaluation(1)]
        if n_estimators is None:
            callbacks.append(lgbm.early_stopping(250))
        if evaluate:
            lgbm_logger.info(f'Evaluation Trial')
        else:
            lgbm_logger.info(f'Trial number : {trial.number}')
            callbacks.append(optuna.integration.LightGBMPruningCallback(trial, 'l2', 'valid_0'))
        
        regression_model = lgbm.LGBMRegressor(**regressor_params)

        regression_model.fit(
            X_t.to_numpy(),
            y_train,
            categorical_feature = cats,
            eval_set = [(X_v.to_numpy(), y_val)],
            eval_metric = 'l2',
            feature_name = columns,
            callbacks = callbacks
        )

        if evaluate:
            lgbm_logger.info(f"Best validation RMSE : {regression_model.best_score_['valid_0']['l2']}")
            lgbm_logger.info(f'In-sample r^2 : {regression_model.score(X_t.to_numpy(), y_train)}')
            lgbm_logger.info(f'Early stopping validation set r^2 : {regression_model.score(X_v.to_numpy(), y_val)}')
            if test_set is not None:
                test_data, test_target = test_set
                lgbm_logger.info(f'Out-of-sample r^2 : {regression_model.score(pipe.fit_transform(test_data).to_numpy(), test_target)}')
            return regression_model

        return regression_model.best_score_['valid_0']['l2']

    return objective

In [ ]:
objective = objective_factory(X_train, replies_train, X_val, replies_val)

In [ ]:
target = 'replies'

optuna.logging.get_logger("optuna").addHandler(logging.FileHandler(f'TreeModels/logs/optuna_target={target}.log'))

# study = optuna.create_study(direction = 'minimize', study_name = f'tree_regressor_for_{target}', storage = f"sqlite:///TreeModels/optuna_study_for_{target}.db", pruner = optuna.pruners.MedianPruner(n_warmup_steps = 200))
study = optuna.create_study(
    direction = 'minimize',
    study_name = f'tree_regressor_for_{target}',
    storage = f"sqlite:///TreeModels/optuna_study_for_{target}.db",
    pruner = optuna.pruners.SuccessiveHalvingPruner(min_resource = 200),
    # sampler = optuna.samplers.TPESampler()
)

In [ ]:
study.optimize(objective, n_trials = 1000)

In [ ]:
# study = optuna.create_study(direction = 'minimize', study_name = f'tree_regressor_for_{target}', storage = f"sqlite:///TreeModels/optuna_study_for_{target}.db", load_if_exists = True)

In [ ]:
def objective_factory(X_train, y_train, X_val, y_val, evaluate = False, test_set = None, n_estimators = None):

    def objective(trial : optuna.Trial):

        TIME_ENCODE_TYPE = 'rbf'
        N_TIME_GAUSSIANS = 4
        SENTIMENT_TYPE = trial.suggest_categorical('sentiment_type', ['none', 'vader', 'distilbert', 'roberta', 'siebert'])
        NTOPICS = _nTopics_list[trial.suggest_int('nTopics_INDEX', 0, len(_nTopics_list) - 1)]
        TOP_5 = trial.suggest_categorical('top_5', [True, False])
        if TOP_5:
            INCLUDE_SCORES = trial.suggest_categorical('include_scores', [True, False])
        else:
            INCLUDE_SCORES = None
        SYMMETRICBETA = trial.suggest_categorical('symmetricbeta', [True, False])

        DROP_MONTH = trial.suggest_categorical('drop_month', [True, False])
        DROP_DAYOFWEEK = trial.suggest_categorical('drop_dayofweek', [True, False])
        DROP_COMMENTS = trial.suggest_categorical('drop_comments', [True, False])
        DROP_LENGTHS = trial.suggest_categorical('drop_lengths', [True, False])

        pipe = Pipeline([
            ('section_encoder', IntegerCategoricalEncoder('section')),
            ('time_encoder', TimeEncoder(time_encode_type = TIME_ENCODE_TYPE, n_time_gaussians = N_TIME_GAUSSIANS)),
            ('drop_columns', ColumnRemover(DROP_MONTH, DROP_DAYOFWEEK, DROP_COMMENTS, DROP_LENGTHS)),
            ('sentiment_selector', SentimentSelector(sentiment_type = SENTIMENT_TYPE)),
            ('topic_loader', TopicLoader(nTopics = NTOPICS, top_5 = TOP_5, include_scores = INCLUDE_SCORES, symmetricbeta = SYMMETRICBETA))
        ])

        X_t = pipe.fit_transform(X_train)
        X_v = pipe.fit_transform(X_val)

        columns = list(X_t.columns)
        cats = [idx for idx, col in enumerate(columns) if categorical_identifier(col)]

        regressor_params = {
            'device_type' : 'cpu',
            'objective' : 'regression',
            'n_estimators' : 999_999 if n_estimators is None else n_estimators,
            'learning_rate' : 0.02,
            'num_leaves' : trial.suggest_int('num_leaves', 4, 2000),
            'max_depth' : trial.suggest_int('max_depth', 2, 30),
            'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 10, 200, step = 10),
            'lambda_l1' : trial.suggest_float('lambda_l1', 1e-6, 0.1, log = True),
            'lambda_l2' : trial.suggest_float('lambda_l2', 1e-6, 0.1, log = True),
            'min_gain_to_split' : trial.suggest_float('min_gain_to_split', 1e-6, 0.1, log = True),
            'bagging_freq' : 1,
            'bagging_fraction' : trial.suggest_float('bagging_fraction', 0.5, 0.99),
            'feature_fraction' : trial.suggest_float('feature_fraction', 0.05, 0.5),
            'min_data_per_group' : trial.suggest_int('min_data_per_group', 1, 10),
            'cat_smooth' : trial.suggest_float('cat_smooth', 0, 50.0),
            'n_jobs': 6,
            'importance_type' : 'gain'
        }

        callbacks = [lgbm.log_evaluation(1)]
        if n_estimators is None:
            callbacks.append(lgbm.early_stopping(250))
        if evaluate:
            lgbm_logger.info(f'Evaluation Trial')
        else:
            lgbm_logger.info(f'Trial number : {trial.number}')
            callbacks.append(optuna.integration.LightGBMPruningCallback(trial, 'l2', 'valid_0'))
        
        regression_model = lgbm.LGBMRegressor(**regressor_params)

        regression_model.fit(
            X_t.to_numpy(),
            y_train,
            categorical_feature = cats,
            eval_set = [(X_v.to_numpy(), y_val)],
            eval_metric = 'l2',
            feature_name = columns,
            callbacks = callbacks
        )

        if evaluate:
            lgbm_logger.info(f"Best validation RMSE : {regression_model.best_score_['valid_0']['l2']}")
            lgbm_logger.info(f'In-sample r^2 : {regression_model.score(X_t.to_numpy(), y_train)}')
            lgbm_logger.info(f'Early stopping validation set r^2 : {regression_model.score(X_v.to_numpy(), y_val)}')
            if test_set is not None:
                test_data, test_target = test_set
                lgbm_logger.info(f'Out-of-sample r^2 : {regression_model.score(pipe.fit_transform(test_data).to_numpy(), test_target)}')
            return regression_model

        return regression_model.best_score_['valid_0']['l2']

    return objective

In [ ]:
objective = objective_factory(X_train, retweets_train, X_val, retweets_val)

In [ ]:
target = 'replies'

study = optuna.create_study(
    direction = 'minimize',
    study_name = f'tree_regressor_for_{target}',
    storage = f"sqlite:///TreeModels/optuna_study_for_{target}.db",
    pruner = optuna.pruners.SuccessiveHalvingPruner(min_resource = 200),
    # sampler = optuna.samplers.TPESampler()
)

In [ ]:
study.optimize(objective, n_trials = 1000)

In [ ]:
def objective_factory(X_train, y_train, X_val, y_val, evaluate = False, test_set = None, n_estimators = None):

    def objective(trial : optuna.Trial):

        TIME_ENCODE_TYPE = 'rbf'
        N_TIME_GAUSSIANS = 4
        SENTIMENT_TYPE = 'roberta'
        NTOPICS = _nTopics_list[trial.suggest_int('nTopics_INDEX', 0, len(_nTopics_list) - 1)]
        TOP_5 = trial.suggest_categorical('top_5', [True, False])
        if TOP_5:
            INCLUDE_SCORES = trial.suggest_categorical('include_scores', [True, False])
        else:
            INCLUDE_SCORES = None
        SYMMETRICBETA = trial.suggest_categorical('symmetricbeta', [True, False])

        DROP_MONTH = False
        DROP_DAYOFWEEK = False
        DROP_COMMENTS = True
        DROP_LENGTHS = False

        pipe = Pipeline([
            ('section_encoder', IntegerCategoricalEncoder('section')),
            ('time_encoder', TimeEncoder(time_encode_type = TIME_ENCODE_TYPE, n_time_gaussians = N_TIME_GAUSSIANS)),
            ('drop_columns', ColumnRemover(DROP_MONTH, DROP_DAYOFWEEK, DROP_COMMENTS, DROP_LENGTHS)),
            ('sentiment_selector', SentimentSelector(sentiment_type = SENTIMENT_TYPE)),
            ('topic_loader', TopicLoader(nTopics = NTOPICS, top_5 = TOP_5, include_scores = INCLUDE_SCORES, symmetricbeta = SYMMETRICBETA))
        ])

        X_t = pipe.fit_transform(X_train)
        X_v = pipe.fit_transform(X_val)

        columns = list(X_t.columns)
        cats = [idx for idx, col in enumerate(columns) if categorical_identifier(col)]

        regressor_params = {
            'device_type' : 'cpu',
            'objective' : 'regression',
            'n_estimators' : 999_999 if n_estimators is None else n_estimators,
            'learning_rate' : 0.02,
            'num_leaves' : trial.suggest_int('num_leaves', 4, 2000),
            'max_depth' : trial.suggest_int('max_depth', 2, 20),
            'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 5, 50, step = 5),
            'lambda_l1' : trial.suggest_float('lambda_l1', 1e-6, 0.1, log = True),
            'lambda_l2' : trial.suggest_float('lambda_l2', 1e-6, 0.1, log = True),
            'min_gain_to_split' : trial.suggest_float('min_gain_to_split', 1e-6, 0.1, log = True),
            'bagging_freq' : 1,
            'bagging_fraction' : 0.95,
            'feature_fraction' : 0.15,
            'min_data_per_group' : trial.suggest_int('min_data_per_group', 1, 15),
            'cat_smooth' : trial.suggest_float('cat_smooth', 0, 100.0),
            'max_cat_threshold' : trial.suggest_int('max_cat_threshold', 1, 50),
            'cat_l2' : trial.suggest_float('cat_l2', 1e-8, 10.0, log = True),
            'max_cat_to_onehot' : trial.suggest_int('max_cat_to_onehot', 1, 10),
            'n_jobs': 6,
            'importance_type' : 'gain'
        }

        callbacks = [lgbm.log_evaluation(1)]
        if n_estimators is None:
            callbacks.append(lgbm.early_stopping(250))
        if evaluate:
            lgbm_logger.info(f'Evaluation Trial')
        else:
            lgbm_logger.info(f'Trial number : {trial.number}')
            callbacks.append(optuna.integration.LightGBMPruningCallback(trial, 'l2', 'valid_0'))
        
        regression_model = lgbm.LGBMRegressor(**regressor_params)

        regression_model.fit(
            X_t.to_numpy(),
            y_train,
            categorical_feature = cats,
            eval_set = [(X_v.to_numpy(), y_val)],
            eval_metric = 'l2',
            feature_name = columns,
            callbacks = callbacks
        )

        if evaluate:
            lgbm_logger.info(f"Best validation RMSE : {regression_model.best_score_['valid_0']['l2']}")
            lgbm_logger.info(f'In-sample r^2 : {regression_model.score(X_t.to_numpy(), y_train)}')
            lgbm_logger.info(f'Early stopping validation set r^2 : {regression_model.score(X_v.to_numpy(), y_val)}')
            if test_set is not None:
                test_data, test_target = test_set
                lgbm_logger.info(f'Out-of-sample r^2 : {regression_model.score(pipe.fit_transform(test_data).to_numpy(), test_target)}')
            return regression_model

        return regression_model.best_score_['valid_0']['l2']

    return objective

In [ ]:
objective = objective_factory(X_train, replies_train, X_val, replies_val)

In [ ]:
target = 'replies'

study = optuna.create_study(
    direction = 'minimize',
    study_name = f'tree_regressor_for_{target}',
    storage = f"sqlite:///TreeModels/optuna_study_for_{target}.db",
    pruner = optuna.pruners.SuccessiveHalvingPruner(min_resource = 200),
    # sampler = optuna.samplers.TPESampler(),
    load_if_exists = True
)

In [ ]:
study.optimize(objective, n_trials = 1000)

In [ ]:
eval_trial = objective_factory(X_train, replies_train, X_val, replies_val, evaluate = True, test_set = (X_test, replies_test))
m = eval_trial(study.best_trial)

In [ ]:
eval_trial = objective_factory(X_combined, retweets_combined, X_combined, retweets_combined, evaluate = True, test_set = (X_test, retweets_test), n_estimators = 963)
m = eval_trial(study.best_trial)

In [ ]:
study.best_params